In [1]:
import nibabel as nii
import numpy as np
import os
import tqdm
import dataloader
import loader_helper
from scipy import ndimage as ndi
from skimage.filters import threshold_otsu
import pickle

In [2]:
train_path = '/home/localadmin/segthor/data/train_resampled'
output_path = '/home/localadmin/segthor/data/train_resampled_cropped'

In [3]:
file_list = [f for f in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, f))]
file_list.sort()
print(file_list)

['Patient_01', 'Patient_02', 'Patient_03', 'Patient_04', 'Patient_05', 'Patient_06', 'Patient_07', 'Patient_08', 'Patient_09', 'Patient_10', 'Patient_11', 'Patient_12', 'Patient_13', 'Patient_14', 'Patient_15', 'Patient_16', 'Patient_17', 'Patient_18', 'Patient_19', 'Patient_20', 'Patient_21', 'Patient_22', 'Patient_23', 'Patient_24', 'Patient_25', 'Patient_26', 'Patient_27', 'Patient_28', 'Patient_29', 'Patient_30', 'Patient_31', 'Patient_32', 'Patient_33', 'Patient_34', 'Patient_35', 'Patient_36', 'Patient_37', 'Patient_38', 'Patient_39', 'Patient_40']


In [4]:
def read_nii(file_path):
    return nii.load(file_path)

In [5]:
for i,f in enumerate(file_list):
    filename = os.path.join(train_path,f,f+'.nii.gz')
    filename_gt = os.path.join(train_path,f,'GT.nii.gz')
    header = read_nii(filename)
    image=np.array(header.get_data()).astype(np.float32)
    gt = np.array(read_nii(filename_gt).get_data())
    
    image_med = ndi.median_filter(image,size=20)
    treshold = threshold_otsu(image_med.reshape(image_med.shape[0],-1))
    image_tr = image_med > treshold
    
    bbox = loader_helper.bbox3(image_tr)
    
    desc = {
        'shape':image.shape,
        'bbox':bbox
    }
    
    print(bbox[0],bbox[1])
    
    image_crop = image[bbox[0,0]:bbox[1,0],bbox[0,1]:bbox[1,1],bbox[0,2]:bbox[1,2]]
    gt_crop = gt[bbox[0,0]:bbox[1,0],bbox[0,1]:bbox[1,1],bbox[0,2]:bbox[1,2]]
    
    #image_norm = dataloader.normalize(image, image > -500)
    #np.save(os.path.join(output_path,f+'.npy'),image_norm)
    os.mkdir(os.path.join(output_path, f))
    new_image = nii.Nifti1Image(image_crop.astype(np.float32), header.affine)
    new_gt = nii.Nifti1Image(gt_crop.astype(np.float32), header.affine)
    
    nii.save(new_image,os.path.join(output_path,f,f+'.nii.gz'))
    nii.save(new_gt,os.path.join(output_path,f,'GT.nii.gz'))
    pickle.dump( desc, open( os.path.join(output_path,f,'desc.p'), 'wb' ) )
    print(f)

[25 99  0] [476 380 304]
Patient_01
[ 8 98  0] [489 399 409]
Patient_02
[  0 143   0] [490 381 244]
Patient_03
[  9 125   0] [462 395 262]
Patient_04
[ 14 124   0] [489 401 378]
Patient_05
[ 16 117   0] [470 399 331]
Patient_06
[26 91  0] [472 363 238]
Patient_07
[ 10 126   0] [481 448 284]
Patient_08
[ 39 118   0] [450 370 256]
Patient_09
[ 0 61  0] [490 374 216]
Patient_10
[ 41 128   0] [590 490 299]
Patient_11
[ 35 117   0] [451 387 227]
Patient_12
[ 8 93  0] [485 375 303]
Patient_13
[ 21 121   0] [446 389 269]
Patient_14
[ 0 85  0] [458 353 249]
Patient_15
[ 0 61  0] [499 397 276]
Patient_16
[ 14 146   0] [461 403 276]
Patient_17
[ 0 98  0] [499 408 304]
Patient_18
[  8 126   0] [493 499 316]
Patient_19
[ 72 162   0] [639 540 342]
Patient_20
[  0 115   0] [493 379 262]
Patient_21
[ 29 112   0] [457 403 301]
Patient_22
[ 18 115   0] [484 371 254]
Patient_23
[46 45  0] [454 395 279]
Patient_24
[  7 130   0] [491 383 249]
Patient_25
[11 35  0] [488 360 328]
Patient_26
[  0 124   0] [4